In [1]:
# !pip install tweepy
# !pip install nltk
# !pip install Sastrawi
# !pip install swifter

   # Pengambilan Data Dari Twitter

In [2]:
import tweepy
import csv
import string
import pandas as pd 

In [3]:
# access_token = "1339862432500498433-bWl7UFBHqQTdKCIbj0GavDCIsnHHeT"
# access_token_secret ="vmP6Eb1WBGIwaGiRUHCLo69OwEzKVbcQOp6iMXBuw99aF"
# consumer_key ="PodLGfK2ky8VbG1mFpEKOFLTQ"
# consumer_secret="0QocN2CdV41mCdUEd5VWxpMjNB4UqCFE0hG6zPwmVOTEuCcf7f"

In [4]:
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tweepy.API(auth)

# def scraptweets(search_words, date_since, date_until):

#     db_tweets = pd.DataFrame(columns=['username', 'tweetcreatedts', 'tweet'])

#     tweets = tweepy.Cursor(
#                     api.search, q=search_words, lang="id", 
#                     since=date_since, until=date_until,  tweet_mode='extended').items(1000)

#     tweet_list = [tweet for tweet in tweets]

#     for tweet in tweet_list:
#         username = tweet.user.screen_name
#         tweetcreatedts = tweet.created_at

#         try:
#             tweet = tweet.retweeted_status.full_text
#         except AttributeError:
#             tweet = tweet.full_text

#         ith_tweet = [username, tweetcreatedts, tweet]

#         db_tweets.loc[len(db_tweets)] = ith_tweet
    
#     print('Proses Scrapping Selesai Dengan Jumlah Data', len(db_tweets))
#     filename = 'dataset.csv'
#     db_tweets.to_csv(filename, index=False)

In [5]:
# search_words = "Daring"
# date_since = "2021-01-01"
# date_until = "2021-05-12"

# scraptweets(search_words, date_since, date_until)

# PREPROCESSING

In [6]:
import pandas as pd 

TWEET_DATA = pd.read_csv("dataset.csv")


In [7]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
TWEET_DATA['tweet_cf'] = TWEET_DATA['tweet'].str.lower()


print('Case Folding Result : \n')
print(TWEET_DATA['tweet_cf'].head(5))
print('\n\n\n')

Case Folding Result : 

0    cerita si ayam - diskon 20% menjadi rp.20000\r...
1    buku aktivitas mengenal hewan - diskon 20% men...
2    seri adab makan dan minum - diskon 20% menjadi...
3    update perkembangan covid-19 di kabupaten luwu...
4    seri adab berbicara - diskon 20% menjadi rp.20...
Name: tweet_cf, dtype: object






In [8]:
import string 
import re #regex library
import preprocessor as p

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
TWEET_DATA['tweet_cf'] = TWEET_DATA['tweet_cf'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

TWEET_DATA['tweet_cf'] = TWEET_DATA['tweet_cf'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

TWEET_DATA['tweet_cf'] = TWEET_DATA['tweet_cf'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

TWEET_DATA['tweet_cf'] = TWEET_DATA['tweet_cf'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

TWEET_DATA['tweet_cf'] = TWEET_DATA['tweet_cf'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

TWEET_DATA['tweet_clean'] = TWEET_DATA['tweet_cf'].apply(remove_singl_char)

def preprocessing_data(x):
    return p.clean(x)

TWEET_DATA['tweet_clean'] = TWEET_DATA['tweet_cf'].apply(preprocessing_data)

def tokenize_data(x):
    return p.tokenize(x)

TWEET_DATA['tweet_clean'] = TWEET_DATA['tweet_clean'].apply(tokenize_data)


In [9]:
alay_dict = pd.read_csv('new_kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 
                                      1: 'replacement'})
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

TWEET_DATA['tweet_clean_alay'] = TWEET_DATA['tweet_clean'].apply(normalize_alay) 
TWEET_DATA.drop_duplicates(keep=False,inplace=True)


In [10]:
import nltk
from nltk.tokenize import word_tokenize 

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

TWEET_DATA['tweet_token'] = TWEET_DATA['tweet_clean_alay'].apply(word_tokenize_wrapper)

In [11]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')
print(len(list_stopwords))

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
len(list_stopwords)
# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwordsID.csv", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
len(list_stopwords)
# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

TWEET_DATA['tweet_stopword'] = TWEET_DATA['tweet_token'].apply(stopwords_removal) 


print(TWEET_DATA['tweet_stopword'])

758
0       [cerita, ayam, diskon, rupiah, gratis, ongkos,...
1       [buku, aktivitas, mengenal, hewan, diskon, rup...
2       [seri, adab, makan, minum, diskon, rupiah, gra...
3       [pembaruan, perkembangan, covid, kabupaten, lu...
4       [seri, adab, berbicara, diskon, rupiah, gratis...
                              ...                        
2476               [kadang, nyatet, pas, daring, tinggal]
2477                             [nyatet, materi, daring]
2479                                             [daring]
2481    [cerita, wkwk, siang, pas, break, daring, menu...
2482    [dipakai, enak, tanggal, april, handphone, ero...
Name: tweet_stopword, Length: 1965, dtype: object


In [12]:
def normalisasi(tweet):
    kamus_slangword = eval(open("combined_slang_words.txt").read()) # Membuka dictionary slangword
    pattern = re.compile(r'\b( ' + '|'.join (kamus_slangword.keys())+r')\b') # Search pola kata (contoh kpn -> kapan)
    content = []
    for kata in tweet:
        filteredSlang = pattern.sub(lambda x: kamus_slangword[x.group()],kata) # Replace slangword berdasarkan pola review yg telah ditentukan
        content.append(filteredSlang.lower())
    tweet = content
    return tweet
TWEET_DATA['tweet_normalisasi'] = TWEET_DATA['tweet_stopword'].apply(normalisasi)
print(TWEET_DATA['tweet_normalisasi'])

0       [cerita, ayam, diskon, rupiah, gratis, ongkos,...
1       [buku, aktivitas, mengenal, hewan, diskon, rup...
2       [seri, adab, makan, minum, diskon, rupiah, gra...
3       [pembaruan, perkembangan, covid, kabupaten, lu...
4       [seri, adab, berbicara, diskon, rupiah, gratis...
                              ...                        
2476               [kadang, nyatet, pas, daring, tinggal]
2477                             [nyatet, materi, daring]
2479                                             [daring]
2481    [cerita, wkwk, siang, pas, break, daring, menu...
2482    [dipakai, enak, tanggal, april, handphone, ero...
Name: tweet_normalisasi, Length: 1965, dtype: object


In [13]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in TWEET_DATA['tweet_normalisasi']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

TWEET_DATA['tweet_stemming'] = TWEET_DATA['tweet_normalisasi'].swifter.apply(get_stemmed_term)
print(TWEET_DATA['tweet_stemming'])

4409
------------------------
cerita : cerita
ayam : ayam
diskon : diskon
rupiah : rupiah
gratis : gratis
ongkos : ongkos
kirim : kirim
ribu : ribu
keseluruh : seluruh
indonesia : indonesia
sinopsisciap : sinopsisciap
assalamualaikum : assalamualaikum
temanteman : temanteman
taji : taji
anak : anak
buku : buku
aktivitas : aktivitas
mengenal : kenal
hewan : hewan
sinopsismewarnai : sinopsismewarnai
membaca : baca
menulis : tulis
menghitung : hitung
seri : seri
adab : adab
makan : makan
minum : minum
sinopsismengapa : sinopsismengapa
bu : bu
ihda : ihda
menegur : tegur
abi : abi
pembaruan : baru
perkembangan : kembang
covid : covid
kabupaten : kabupaten
luwu : luwu
timur : timur
april : april
sembuh : sembuh
perbaharui : baharu
informasi : informasi
berbicara : bicara
sinopsissaat : sinopsissaat
gema : gema
gina : gina
berbohong : bohong
indahnya : indah
bepergian : pergi
sinopsislila : sinopsislila
kesal : kesal
lupa : lupa
membawa : bawa
mainan : main
pasirnya : pasir
pantai : pantai
b

dipahami : paham
gaefektif : gaefektif
menurutku : turut
menonton : tonton
lebar : lebar
menghadapi : hadap
tipe : tipe
langsung : langsung
bingung : bingung
psikopat : psikopat
selsai : selsai
eh : eh
diajak : ajak
hukum : hukum
pecahkan : pecah
suuzan : suuzan
kwanan : kwanan
casper : casper
ngusiri : ngusiri
kdrama : kdrama
ongoing : ongoing
katadata : katadata
mellow : mellow
kakak : kakak
laki : laki
butuhnya : butuh
perhatian : perhati
ekstraa : ekstraa
jarak : jarak
menimbulkan : timbul
unicef : unicef
menemukan : temu
keluarga : keluarga
miskin : miskin
pola : pola
tidur : tidur
kacau : kacau
jurusankuti : jurusankuti
nyaman : nyaman
jurusan : jurus
prioritas : prioritas
admin : admin
iphone : iphone
satuan : satu
air : air
sediakan : sedia
internetgratis : internetgratis
menang : menang
butuh : butuh
serba : serba
karuan : karuan
miris : miris
murid : murid
terpaksa : paksa
pinggirselokan : pinggirselokan
meja : meja
gaperna : gaperna
anime : anime
komik : komik
novel : novel


pikirandan : pikirandan
dengarkan : dengar
sistemnya : sistem
open : open
booking : booking
krasa : krasa
setuju : tuju
spp : spp
nakalnya : nakal
wajar : wajar
niat : niat
dajjal : dajjal
done : done
daringkarena : daringkarena
hpku : hpku
bngtaamiin : bngtaamiin
aduh : aduh
gawat : gawat
ratusan : ratus
karanganyar : karanganyar
ramerame : ramerame
pilih : pilih
berhenti : henti
do : do
tanyakenapa : tanyakenapa
universitas : universitas
impian : impi
ngandelin : ngandelin
twit : twit
lumayan : lumayan
dibagian : bagi
hitungan : hitung
wangsit : wangsit
pick : pick
egx : egx
kursi : kursi
dilantai : lantai
diamati : amat
ah : ah
chill : chill
kegiatan belajar mengajar : giat ajar ajar
offline : offline
asyik : asyik
temannya : teman
darling : darling
pekerjaanku : kerja
terganggu : ganggu
hapalan : hapalan
hadeeeeh : hadeeeeh
ahok : ahok
ahoker : ahoker
terhormat : hormat
pertamina : pertamina
simprug : simprug
bayar : bayar
biaya : biaya
ukt : ukt
cuti : cuti
kampus : kampus
mengang

diamond : diamond
bismilahirohmanirohim : bismilahirohmanirohim
bunda : bunda
adiku : adi
bahagia : bahagia
lancarkan : lancar
rezekya : rezekya
pusaran : pusar
konflik : konflik
perebutan : rebut
lahan : lahan
memanggil : panggil
solidaritas : solidaritas
kaum : kaum
peluncuran : luncur
diselenggarakan : selenggara
barat : barat
silakan : sila
mendaftar : daftar
gatauuuu : gatauuuu
coronconggg : coronconggg
merasakan : rasa
harga : harga
melambung : lambung
belajardaring : belajardaring
ltshare : ltshare
appreciategt : appreciategt
kiatkiat : kiatkiat
mencapai : capai
efektivitas : efektivitas
situ : situ
melatih : latih
caramu : cara
mayoritas : mayoritas
isi : isi
kuesioner : kuesioner
rep : rep
direct : direct
message : message
walah : walah
mangat : mangat
sel : sel
wafat : wafat
pekan : pekan
perawatan : awat
karyawan : karyawan
ngeri : ngeri
bayangkan : bayang
rael : rael
awali : awal
pagimu : pagi
senyum : senyum
puasanya : puasa
cantik : cantik
hv : hv
wonderfull : wonderfull


apni : apni
bsdk : bsdk
depresi : depresi
allhamdulillah : allhamdulillah
gih : gih
daringlt : daringlt
ekonomi : ekonomi
santay : santay
hdeh : hdeh
ntaran : ntaran
seninngk : seninngk
pertemuan : temu
kahh : kahh
auto : auto
astaga : astaga
hafal : hafal
conference : conference
azan : azan
paipaiii : paipaiii
duain : duain
laptopnya : laptop
gila : gila
spaces : spaces
berjudul : judul
ngelanjutin : ngelanjutin
pembahasan : bahas
seputar : putar
biologi : biologi
intensif : intensif
gessemangat : gessemangat
dihari : hari
malesjaga : malesjaga
sakitistirahat : sakitistirahat
monday : monday
tmne : tmne
kuch : kuch
tha : tha
kar : kar
lia : lia
pgie : pgie
hdue : hdue
insta : insta
follow : follow
requestugalai : requestugalai
accept : accept
panavum : panavum
makkaleh : makkaleh
validitupun : validitupun
kepaksa : paksa
pendapatan : dapat
toko : toko
oren : oren
puluhan : puluh
jutabulan : jutabulan
prakerja : prakerja
jb : jb
aisa : aisa
khelega : khelega
deserve : deserve
karta : k

nolong : nolong
dulunya : dulunya
ngejajah : ngejajah
terpikirkan : pikir
gra : gra
twettan : twettan
profil : profil
lengkap : lengkap
ngenalinnya : ngenalinnya
butuuuuh : butuuuuh
babayy : babayy
perut : perut
seca : seca
lucu : lucu
wisuda : wisuda
seangkatan : angkat
keras : keras
memuaskan : muas
but : but
renggang : renggang
hubungan : hubung
tvt : tvt
daringgw : daringgw
lagie : lagie
kilat : kilat
must : must
come : come
daringd : daringd
mars : mars
lemesnya : lemes
maria : maria
meninggalkan : tinggal
aula : aula
asrama : asrama
tinggalkan : tinggal
melambaikan : lambai
tangannya : tangan
arah : arah
sir : sir
jordan : jordan
bergumam : gumam
bye : bye
berjumpa : jumpa
coming : coming
berjalan : jalan
hihii : hihii
babelt : babelt
hwhwhw : hwhwhw
smartren : smartren
ceramah : ceramah
resume : resume
wdland : wdland
momong : momong
pembelaan : bela
pejuang : juang
sarjana : sarjana
perkuliahan : kuliah
eks : eks
members : members
perwakilannya : wakil
deng : deng
diundang : un

rektorat : rektorat
putusan : putus
resmi : resmi
berbeda : beda
pemira : pemira
serentak : serentak
polines : polines
terselenggara : selenggara
seninsabtu : seninsabtu
klik : klik
rekanrekannya : rekanrekannya
ceritakan : cerita
keseruan : seru
ssnin : ssnin
hasemelh : hasemelh
lusa : lusa
presiden : presiden
peny : peny
bowling : bowling
control : control
panunga : panunga
deiii : deiii
thakur : thakur
una : una
tahan : tahan
jaddduuuuuu : jaddduuuuuu
thongi : thongi
elunthu : elunthu
match : match
pota : pota
jaddu : jaddu
jelang : jelang
restui : restu
ikuti : ikut
dips : dips
tapiii : tapiii
berprogres : progres
mahasiswimahasiswa : mahasiswimahasiswa
gencet : gencet
smasmk : smasmk
okok : okok
cantikganteng : cantikganteng
panitia : panitia
hero : hero
emel : emel
buhet : buhet
sopan : sopan
santunya : santunya
pulkam : pulkam
cumlaude : cumlaude
mapres : mapres
beso : beso
termotivasi : motivasi
taufik : taufik
ketua : ketua
hima : hima
administrasi : administrasi
unmul : unmul

klubklub : klubklub
boikot : boikot
kekerasandaring : kekerasandaring
respon : respon
berlanjutnya : lanjut
pemain : main
sepak : sepak
bola : bola
inggri : inggri
perdata : perdata
gas : gas
maunya : mau
bhabin : bhabin
cilacap : cilacap
siaga : siaga
bhayangkara : bhayangkara
pembina : bina
keamanan : aman
ketertiban : tertib
brigadir : brigadir
tri : tri
setya : setya
dialogis : dialogis
dayeuhluhur : dayeuhluhur
pegadaiannya : gadai
klaim : klaim
lelang : lelang
posnya : pos
bahut : bahut
miya : miya
besk : besk
ai : ai
sekolaggg : sekolaggg
silaturahmi : silaturahmi
joshi : joshi
ohho : ohho
jyada : jyada
semau : mau
dosbing : dosbing
okey : okey
trisya : trisya
walo : walo
judulnya : judul
peradilan : adil
omelan : omel
telinga : telinga
kanan : kanan
mempan : mempan
tumben : tumben
repot : repot
bawabawa : bawabawa
moral : moral
hiya : hiya
garis : garis
sistemdaring : sistemdaring
data : data
everyone : everyone
their : their
own : own
matter : matter
were : were
hiringbcos : h

wajibapa : wajibapa
guruny : guruny
adaaa : adaaa
membicarakan : bicara
plastij : plastij
wallpaper : wallpaper
exo : exo
haji : haji
mabrur : mabrur
thawaf : thawaf
muterin : muterin
inti : inti
mengunjungi : unjung
kot : kot
daringkalo : daringkalo
carik : carik
bosss : bosss
pegawai : pegawai
akxkakksksks : akxkakksksks
auu : auu
keselllll : keselllll
mi : mi
aleksemenjak : aleksemenjak
berinteraksi : interaksi
orgskrng : orgskrng
omong : omong
pemalu : malu
grogian : grogi
masalu : masalu
loginv : loginv
chi : chi
ponga : ponga
enaku : enaku
vekama : vekama
iruki : iruki
content : content
eyyy : eyyy
venam : venam
apdiyey : apdiyey
pesitey : pesitey
fulltime : fulltime
memudahkan : mudah
wargi : wargi
grati : grati
manfaatkan : manfaat
kondisimu : kondisi
berhak : hak
jabaar : jabaar
diterbitkan : terbit
koleksi : koleksi
peta : peta
lawas : lawas
belanda : belanda
apalah : apa
memikul : pikul
ospeknya : ospeknya
trbs : trbs
dikangenin : dikangenin
pata : pata
berkutat : kutat
meny


0       [cerita, ayam, diskon, rupiah, gratis, ongkos,...
1       [buku, aktivitas, kenal, hewan, diskon, rupiah...
2       [seri, adab, makan, minum, diskon, rupiah, gra...
3       [baru, kembang, covid, kabupaten, luwu, timur,...
4       [seri, adab, bicara, diskon, rupiah, gratis, o...
                              ...                        
2476               [kadang, nyatet, pas, daring, tinggal]
2477                             [nyatet, materi, daring]
2479                                             [daring]
2481    [cerita, wkwk, siang, pas, break, daring, tung...
2482    [pakai, enak, tanggal, april, handphone, eror,...
Name: tweet_stemming, Length: 1965, dtype: object


In [14]:
df = pd.DataFrame(TWEET_DATA)
df['tweet_stemming'] = df['tweet_stemming'].apply(str)

In [15]:
import ast

def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
TWEET_DATA["tweet_join"] = TWEET_DATA["tweet_stemming"].apply(join_text_list)



# LEXICON

In [16]:
#sentimen Lexicon
#Determine sentiment polarity of tweets using indonesia sentiment lexicon (source : https://github.com/fajri91/InSet)

# Loads lexicon positive and negative data
lexicon_positive = dict()
import csv
with open('lexicon_positive.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])

lexicon_negative = dict()
import csv
with open('lexicon_negative.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])
        
# Function to determine sentiment polarity of tweets        
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:
    score = 0
    for word in text:
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
    for word in text:
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
    polarity=''
    if (score > 0):
        polarity = 'positif'
    elif (score < 0):
        polarity = 'negatif'
    else:
        polarity = 'netral'
    return score, polarity

results = TWEET_DATA['tweet_normalisasi'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
TWEET_DATA['jumlah_polarity'] = results[0]
TWEET_DATA['sentimen'] = results[1]
print(TWEET_DATA['sentimen'].value_counts())

positif    759
netral     653
negatif    553
Name: sentimen, dtype: int64


In [17]:
TWEET_DATA

,tweet,tweet_cf,tweet_clean,tweet_clean_alay,tweet_token,tweet_stopword,tweet_normalisasi,tweet_stemming,tweet_join,jumlah_polarity,sentimen
0,Cerita Si Ayam - Diskon 20% menjadi Rp.20000\r...,cerita si ayam diskon menjadi rp gratis ongkos...,cerita si ayam diskon menjadi rp gratis ongkos...,cerita sih ayam diskon menjadi rupiah gratis o...,"[cerita, sih, ayam, diskon, menjadi, rupiah, g...","[cerita, ayam, diskon, rupiah, gratis, ongkos,...","[cerita, ayam, diskon, rupiah, gratis, ongkos,...","['cerita', 'ayam', 'diskon', 'rupiah', 'gratis...",cerita ayam diskon rupiah gratis ongkos kirim ...,5,positif
1,Buku Aktivitas Mengenal Hewan - Diskon 20% men...,buku aktivitas mengenal hewan diskon menjadi r...,buku aktivitas mengenal hewan diskon menjadi r...,buku aktivitas mengenal hewan diskon menjadi r...,"[buku, aktivitas, mengenal, hewan, diskon, men...","[buku, aktivitas, mengenal, hewan, diskon, rup...","[buku, aktivitas, mengenal, hewan, diskon, rup...","['buku', 'aktivitas', 'kenal', 'hewan', 'disko...",buku aktivitas kenal hewan diskon rupiah grati...,13,positif
2,Seri Adab Makan dan Minum - Diskon 20% menjadi...,seri adab makan dan minum diskon menjadi rp gr...,seri adab makan dan minum diskon menjadi rp gr...,seri adab makan dan minum diskon menjadi rupia...,"[seri, adab, makan, dan, minum, diskon, menjad...","[seri, adab, makan, minum, diskon, rupiah, gra...","[seri, adab, makan, minum, diskon, rupiah, gra...","['seri', 'adab', 'makan', 'minum', 'diskon', '...",seri adab makan minum diskon rupiah gratis ong...,11,positif
3,Update Perkembangan Covid-19 di Kabupaten Luwu...,update perkembangan covid di kabupaten luwu ti...,update perkembangan covid di kabupaten luwu ti...,pembaruan perkembangan covid di kabupaten luwu...,"[pembaruan, perkembangan, covid, di, kabupaten...","[pembaruan, perkembangan, covid, kabupaten, lu...","[pembaruan, perkembangan, covid, kabupaten, lu...","['baru', 'kembang', 'covid', 'kabupaten', 'luw...",baru kembang covid kabupaten luwu timur april ...,5,positif
4,Seri Adab Berbicara - Diskon 20% menjadi Rp.20...,seri adab berbicara diskon menjadi rp gratis o...,seri adab berbicara diskon menjadi rp gratis o...,seri adab berbicara diskon menjadi rupiah grat...,"[seri, adab, berbicara, diskon, menjadi, rupia...","[seri, adab, berbicara, diskon, rupiah, gratis...","[seri, adab, berbicara, diskon, rupiah, gratis...","['seri', 'adab', 'bicara', 'diskon', 'rupiah',...",seri adab bicara diskon rupiah gratis ongkos k...,2,positif
...,...,...,...,...,...,...,...,...,...,...,...
2476,"@jsminevly adaa, kadang nyatet duluan jd pas d...",adaa kadang nyatet duluan jd pas daring tingga...,adaa kadang nyatet duluan jd pas daring tingga...,ada kadang nyatet lebih dahulu jadi pas daring...,"[ada, kadang, nyatet, lebih, dahulu, jadi, pas...","[kadang, nyatet, pas, daring, tinggal]","[kadang, nyatet, pas, daring, tinggal]","['kadang', 'nyatet', 'pas', 'daring', 'tinggal']",kadang nyatet pas daring tinggal,4,positif
2477,kalian ada nyatet materi selama daring?,kalian ada nyatet materi selama daring,kalian ada nyatet materi selama daring,kalian ada nyatet materi selama daring,"[kalian, ada, nyatet, materi, selama, daring]","[nyatet, materi, daring]","[nyatet, materi, daring]","['nyatet', 'materi', 'daring']",nyatet materi daring,0,netral
2479,@roxiesxt yaudah daring ajaa,yaudah daring ajaa,yaudah daring ajaa,ya sudah daring saja,"[ya, sudah, daring, saja]",[daring],[daring],['daring'],daring,0,netral
2481,Mau cerita nih wkwkwk\r\nJadi tadi siang pas a...,mau cerita nih wkwkwk jadi tadi siang pas aku ...,mau cerita nih wkwkwk jadi tadi siang pas aku ...,mau cerita nih wkwk jadi tadi siang pas aku br...,"[mau, cerita, nih, wkwk, jadi, tadi, siang, pa...","[cerita, wkwk, siang, pas, break, daring, menu...","[cerita, wkwk, siang, pas, break, daring, menu...","['cerita', 'wkwk', 'siang', 'pas', 'break', 'd...",cerita wkwk siang pas break daring tunggu dose...,5,positif


# PEMBOBOTAN KATA

In [18]:
# convert list formated string to list
import ast
import numpy as np
index = 1

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]

TWEET_DATA["tweet_list"] = TWEET_DATA["tweet_stemming"].apply(convert_text_list)


print(TWEET_DATA["tweet_list"][index])

print("\ntype : ", type(TWEET_DATA["tweet_list"]))

['buku', 'aktivitas', 'kenal', 'hewan', 'diskon', 'rupiah', 'gratis', 'ongkos', 'kirim', 'ribu', 'seluruh', 'indonesia', 'sinopsismewarnai', 'baca', 'tulis', 'hitung', 'buku']

type :  <class 'pandas.core.series.Series'>


In [19]:
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

TWEET_DATA["TF_dict"] = TWEET_DATA['tweet_list'].apply(calc_TF)

TWEET_DATA["TF_dict"].head()

0    {'cerita': 0.06666666666666667, 'ayam': 0.0666...
1    {'buku': 0.11764705882352941, 'aktivitas': 0.0...
2    {'seri': 0.05263157894736842, 'adab': 0.052631...
3    {'baru': 0.1, 'kembang': 0.1, 'covid': 0.1, 'k...
4    {'seri': 0.058823529411764705, 'adab': 0.05882...
Name: TF_dict, dtype: object

In [20]:
print('%20s' % "term", "\t", "TF\n")
for key in TWEET_DATA["TF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key])

                term 	 TF

                buku 	 0.11764705882352941
           aktivitas 	 0.058823529411764705
               kenal 	 0.058823529411764705
               hewan 	 0.058823529411764705
              diskon 	 0.058823529411764705
              rupiah 	 0.058823529411764705
              gratis 	 0.058823529411764705
              ongkos 	 0.058823529411764705
               kirim 	 0.058823529411764705
                ribu 	 0.058823529411764705
             seluruh 	 0.058823529411764705
           indonesia 	 0.058823529411764705
    sinopsismewarnai 	 0.058823529411764705
                baca 	 0.058823529411764705
               tulis 	 0.058823529411764705
              hitung 	 0.058823529411764705


In [21]:
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(TWEET_DATA["TF_dict"])

In [22]:
n_document = len(TWEET_DATA)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)

In [23]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
TWEET_DATA["TF-IDF_dict"] = TWEET_DATA["TF_dict"].apply(calc_TF_IDF)

In [24]:
print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in TWEET_DATA["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key] ,"\t" , TWEET_DATA["TF-IDF_dict"][index][key])

                term 	         TF 	              TF-IDF

                buku 	 0.11764705882352941 	 0.5044012539175332
           aktivitas 	 0.058823529411764705 	 0.31062720184172443
               kenal 	 0.058823529411764705 	 0.29519400981422494
               hewan 	 0.058823529411764705 	 0.3237532930955015
              diskon 	 0.058823529411764705 	 0.2262104650599996
              rupiah 	 0.058823529411764705 	 0.2262104650599996
              gratis 	 0.058823529411764705 	 0.2248263181535176
              ongkos 	 0.058823529411764705 	 0.2262104650599996
               kirim 	 0.058823529411764705 	 0.21959411309372373
                ribu 	 0.058823529411764705 	 0.2234739935520647
             seluruh 	 0.058823529411764705 	 0.227627968094062
           indonesia 	 0.058823529411764705 	 0.20143320240845233
    sinopsismewarnai 	 0.058823529411764705 	 0.3406757679456062
                baca 	 0.058823529411764705 	 0.23864041174630587
               tulis 	 0.05882

In [25]:
# sort descending by value for DF dictionary 
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

TWEET_DATA["TF_IDF_Vec"] = TWEET_DATA["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(TWEET_DATA["TF_IDF_Vec"][0])

print("\nmatrix size : ", len(TWEET_DATA["TF_IDF_Vec"][0]))

print first row matrix TF_IDF_Vec Series

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21861216207327946, 0.0, 0.2282909627295793, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24887332817288688, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25327052602567335, 0.0, 0.0, 0.0, 0.0, 0.2548031605739866, 0.0, 0.25637186040133286, 0.25637186040133286, 0.25637186040133286, 0.2579783638399369, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

matrix size :  50


In [26]:
# Convert Series to List
TF_IDF_Vec_List = np.array(TWEET_DATA["TF_IDF_Vec"].to_list())

# Sum element vector in axis=0 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)

,term,rank
2,besok,116.564118
6,semangat,83.525239
1,ajar,70.790851
7,tidur,63.359697
3,sekolah,59.337556
4,saya,56.515716
8,iya,54.261177
32,darling,48.995786
0,daring,46.808737
5,banget,45.324537


In [27]:
TWEET_DATA

,tweet,tweet_cf,tweet_clean,tweet_clean_alay,tweet_token,tweet_stopword,tweet_normalisasi,tweet_stemming,tweet_join,jumlah_polarity,sentimen,tweet_list,TF_dict,TF-IDF_dict,TF_IDF_Vec
0,Cerita Si Ayam - Diskon 20% menjadi Rp.20000\r...,cerita si ayam diskon menjadi rp gratis ongkos...,cerita si ayam diskon menjadi rp gratis ongkos...,cerita sih ayam diskon menjadi rupiah gratis o...,"[cerita, sih, ayam, diskon, menjadi, rupiah, g...","[cerita, ayam, diskon, rupiah, gratis, ongkos,...","[cerita, ayam, diskon, rupiah, gratis, ongkos,...","['cerita', 'ayam', 'diskon', 'rupiah', 'gratis...",cerita ayam diskon rupiah gratis ongkos kirim ...,5,positif,"[cerita, ayam, diskon, rupiah, gratis, ongkos,...","{'cerita': 0.06666666666666667, 'ayam': 0.0666...","{'cerita': 0.3025816724386625, 'ayam': 0.33988...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Buku Aktivitas Mengenal Hewan - Diskon 20% men...,buku aktivitas mengenal hewan diskon menjadi r...,buku aktivitas mengenal hewan diskon menjadi r...,buku aktivitas mengenal hewan diskon menjadi r...,"[buku, aktivitas, mengenal, hewan, diskon, men...","[buku, aktivitas, mengenal, hewan, diskon, rup...","[buku, aktivitas, mengenal, hewan, diskon, rup...","['buku', 'aktivitas', 'kenal', 'hewan', 'disko...",buku aktivitas kenal hewan diskon rupiah grati...,13,positif,"[buku, aktivitas, kenal, hewan, diskon, rupiah...","{'buku': 0.11764705882352941, 'aktivitas': 0.0...","{'buku': 0.5044012539175332, 'aktivitas': 0.31...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Seri Adab Makan dan Minum - Diskon 20% menjadi...,seri adab makan dan minum diskon menjadi rp gr...,seri adab makan dan minum diskon menjadi rp gr...,seri adab makan dan minum diskon menjadi rupia...,"[seri, adab, makan, dan, minum, diskon, menjad...","[seri, adab, makan, minum, diskon, rupiah, gra...","[seri, adab, makan, minum, diskon, rupiah, gra...","['seri', 'adab', 'makan', 'minum', 'diskon', '...",seri adab makan minum diskon rupiah gratis ong...,11,positif,"[seri, adab, makan, minum, diskon, rupiah, gra...","{'seri': 0.05263157894736842, 'adab': 0.052631...","{'seri': 0.22565319254205432, 'adab': 0.231852...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Update Perkembangan Covid-19 di Kabupaten Luwu...,update perkembangan covid di kabupaten luwu ti...,update perkembangan covid di kabupaten luwu ti...,pembaruan perkembangan covid di kabupaten luwu...,"[pembaruan, perkembangan, covid, di, kabupaten...","[pembaruan, perkembangan, covid, kabupaten, lu...","[pembaruan, perkembangan, covid, kabupaten, lu...","['baru', 'kembang', 'covid', 'kabupaten', 'luw...",baru kembang covid kabupaten luwu timur april ...,5,positif,"[baru, kembang, covid, kabupaten, luwu, timur,...","{'baru': 0.1, 'kembang': 0.1, 'covid': 0.1, 'k...","{'baru': 0.5503805982623525, 'kembang': 0.5098...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Seri Adab Berbicara - Diskon 20% menjadi Rp.20...,seri adab berbicara diskon menjadi rp gratis o...,seri adab berbicara diskon menjadi rp gratis o...,seri adab berbicara diskon menjadi rupiah grat...,"[seri, adab, berbicara, diskon, menjadi, rupia...","[seri, adab, berbicara, diskon, rupiah, gratis...","[seri, adab, berbicara, diskon, rupiah, gratis...","['seri', 'adab', 'bicara', 'diskon', 'rupiah',...",seri adab bicara diskon rupiah gratis ongkos k...,2,positif,"[seri, adab, bicara, diskon, rupiah, gratis, o...","{'seri': 0.058823529411764705, 'adab': 0.05882...","{'seri': 0.2522006269587666, 'adab': 0.2591290...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2476,"@jsminevly adaa, kadang nyatet duluan jd pas d...",adaa kadang nyatet duluan jd pas daring tingga...,adaa kadang nyatet duluan jd pas daring tingga...,ada kadang nyatet lebih dahulu jadi pas daring...,"[ada, kadang, nyatet, lebih, dahulu, jadi, pas...","[kadang, nyatet, pas, daring, tinggal]","[kadang, nyatet, pas, daring, tinggal]","['kadang', 'nyatet'

# SENTIMEN K-NN

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
import numpy as np

komentar= TWEET_DATA["tweet_join"]

case_fold = []
for i in komentar:
    if i not in list_stopwords:
        lowercase = i.translate(str.maketrans("","",string.punctuation)).lower()
        hapus_angka = re.sub(r"\d+", "", lowercase)
        hapus_space = hapus_angka.strip()
        case_fold.append(hapus_space)
print("Hasil Preprocessing : \n",case_fold)

Hasil Preprocessing : 
 ['cerita ayam diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsisciap assalamualaikum temanteman taji anak', 'buku aktivitas kenal hewan diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsismewarnai baca tulis hitung buku', 'seri adab makan minum diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsismengapa bu ihda tegur abi abi makan', 'baru kembang covid kabupaten luwu timur april sembuh baharu informasi', 'seri adab bicara diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsissaat bicara gema gina bohong indah', 'seri adab pergi diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsislila kesal lupa bawa main pasir pantai', 'seri adab ajar diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsisnilai ulang hari jelek angka', 'cerita ayam diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsisciap assalamualaikum temanteman taji anak', 'setan takut diskon rupiah gratis ongkos kirim rib

In [29]:
#Pembobotan
vectorizer = TfidfVectorizer(stop_words=list_stopwords)
x_train = vectorizer.fit_transform(case_fold)
features = vectorizer.get_feature_names()
dense = x_train.todense()
listfeatures = dense.tolist()

df = pd.DataFrame(listfeatures, columns=features)
print("Hasil Pembobotan : ")
df.head(160)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Hasil Pembobotan : 


,aaneh,aap,aauw,ab,abadi,abang,abdi,abe,abi,abisssssss,...,zha,zindagi,zizha,zona,zoom,zoomgmeet,zoomkan,zuhur,zuzur,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.481598,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Label untuk data training
y_train = np.array(TWEET_DATA['sentimen'])
# Clustering the document with KNN classifier
classifier = KNeighborsClassifier()

classifier.fit(x_train,y_train)

KNeighborsClassifier()

In [31]:
testing_kalimat = TWEET_DATA['tweet_join']
bersihkan_kalimat = []
for test in testing_kalimat:
    if test not in list_stopwords:
        lowercase_test = test.translate(str.maketrans("","",string.punctuation)).lower()
        hapus_angka_test = re.sub(r"\d+", "", lowercase_test)
        hapus_space_test = hapus_angka_test.strip()
        bersihkan_kalimat.append(hapus_space_test)
print("Preprocessing Komentar Test : \n",bersihkan_kalimat)

Preprocessing Komentar Test : 
 ['cerita ayam diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsisciap assalamualaikum temanteman taji anak', 'buku aktivitas kenal hewan diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsismewarnai baca tulis hitung buku', 'seri adab makan minum diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsismengapa bu ihda tegur abi abi makan', 'baru kembang covid kabupaten luwu timur april sembuh baharu informasi', 'seri adab bicara diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsissaat bicara gema gina bohong indah', 'seri adab pergi diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsislila kesal lupa bawa main pasir pantai', 'seri adab ajar diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsisnilai ulang hari jelek angka', 'cerita ayam diskon rupiah gratis ongkos kirim ribu seluruh indonesia sinopsisciap assalamualaikum temanteman taji anak', 'setan takut diskon rupiah gratis ongkos k

In [32]:
x_test = vectorizer.transform(bersihkan_kalimat) 
y_pred = classifier.predict(x_test)
y_test = np.array(TWEET_DATA['sentimen'])

In [33]:
def sentiment_analysis_KNN(text):
    a =''
    b =''
    for i in range(len(text)):
        a =  y_pred[i]
        b =  y_test[i]
        
    return a, b
results = testing_kalimat.apply(sentiment_analysis_KNN)
results = list(zip(*results))
TWEET_DATA['hasil_KNN'] = results[0]
print ("\n-------------------------------Sentiment BY KNN------------------------------------------")
print(TWEET_DATA['hasil_KNN'].value_counts())


-------------------------------Sentiment BY KNN------------------------------------------
positif    1433
negatif     461
netral       71
Name: hasil_KNN, dtype: int64


In [34]:
TWEET_DATA

,tweet,tweet_cf,tweet_clean,tweet_clean_alay,tweet_token,tweet_stopword,tweet_normalisasi,tweet_stemming,tweet_join,jumlah_polarity,sentimen,tweet_list,TF_dict,TF-IDF_dict,TF_IDF_Vec,hasil_KNN
0,Cerita Si Ayam - Diskon 20% menjadi Rp.20000\r...,cerita si ayam diskon menjadi rp gratis ongkos...,cerita si ayam diskon menjadi rp gratis ongkos...,cerita sih ayam diskon menjadi rupiah gratis o...,"[cerita, sih, ayam, diskon, menjadi, rupiah, g...","[cerita, ayam, diskon, rupiah, gratis, ongkos,...","[cerita, ayam, diskon, rupiah, gratis, ongkos,...","['cerita', 'ayam', 'diskon', 'rupiah', 'gratis...",cerita ayam diskon rupiah gratis ongkos kirim ...,5,positif,"[cerita, ayam, diskon, rupiah, gratis, ongkos,...","{'cerita': 0.06666666666666667, 'ayam': 0.0666...","{'cerita': 0.3025816724386625, 'ayam': 0.33988...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",positif
1,Buku Aktivitas Mengenal Hewan - Diskon 20% men...,buku aktivitas mengenal hewan diskon menjadi r...,buku aktivitas mengenal hewan diskon menjadi r...,buku aktivitas mengenal hewan diskon menjadi r...,"[buku, aktivitas, mengenal, hewan, diskon, men...","[buku, aktivitas, mengenal, hewan, diskon, rup...","[buku, aktivitas, mengenal, hewan, diskon, rup...","['buku', 'aktivitas', 'kenal', 'hewan', 'disko...",buku aktivitas kenal hewan diskon rupiah grati...,13,positif,"[buku, aktivitas, kenal, hewan, diskon, rupiah...","{'buku': 0.11764705882352941, 'aktivitas': 0.0...","{'buku': 0.5044012539175332, 'aktivitas': 0.31...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",negatif
2,Seri Adab Makan dan Minum - Diskon 20% menjadi...,seri adab makan dan minum diskon menjadi rp gr...,seri adab makan dan minum diskon menjadi rp gr...,seri adab makan dan minum diskon menjadi rupia...,"[seri, adab, makan, dan, minum, diskon, menjad...","[seri, adab, makan, minum, diskon, rupiah, gra...","[seri, adab, makan, minum, diskon, rupiah, gra...","['seri', 'adab', 'makan', 'minum', 'diskon', '...",seri adab makan minum diskon rupiah gratis ong...,11,positif,"[seri, adab, makan, minum, diskon, rupiah, gra...","{'seri': 0.05263157894736842, 'adab': 0.052631...","{'seri': 0.22565319254205432, 'adab': 0.231852...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",positif
3,Update Perkembangan Covid-19 di Kabupaten Luwu...,update perkembangan covid di kabupaten luwu ti...,update perkembangan covid di kabupaten luwu ti...,pembaruan perkembangan covid di kabupaten luwu...,"[pembaruan, perkembangan, covid, di, kabupaten...","[pembaruan, perkembangan, covid, kabupaten, lu...","[pembaruan, perkembangan, covid, kabupaten, lu...","['baru', 'kembang', 'covid', 'kabupaten', 'luw...",baru kembang covid kabupaten luwu timur april ...,5,positif,"[baru, kembang, covid, kabupaten, luwu, timur,...","{'baru': 0.1, 'kembang': 0.1, 'covid': 0.1, 'k...","{'baru': 0.5503805982623525, 'kembang': 0.5098...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",positif
4,Seri Adab Berbicara - Diskon 20% menjadi Rp.20...,seri adab berbicara diskon menjadi rp gratis o...,seri adab berbicara diskon menjadi rp gratis o...,seri adab berbicara diskon menjadi rupiah grat...,"[seri, adab, berbicara, diskon, menjadi, rupia...","[seri, adab, berbicara, diskon, rupiah, gratis...","[seri, adab, berbicara, diskon, rupiah, gratis...","['seri', 'adab', 'bicara', 'diskon', 'rupiah',...",seri adab bicara diskon rupiah gratis ongkos k...,2,positif,"[seri, adab, bicara, diskon, rupiah, gratis, o...","{'seri': 0.058823529411764705, 'adab': 0.05882...","{'seri': 0.2522006269587666, 'adab': 0.2591290...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",positif
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2476,"@jsminevly adaa, kadang nyatet duluan jd pas d...",adaa kadang nyatet duluan jd pas daring tingga...,adaa kadang nyatet duluan jd pas daring tingga...,ada kadang nyatet lebih dahulu jadi pas daring...,"[ada, kadang, nyatet, lebih, dahulu, jadi, pas...","[kadang, nyatet, pas, daring, tinggal]","[kadan

In [35]:
TWEET_DATA.to_excel("hasil-klasifikasi.xlsx")